# 3 Methods for Hyperparameter Tuning with XGBoost

In this notebook, we'll explore 3 different approaches for hyperparameter tuning with XGBoost. These will include:

1. Grid Search
2. Randomized Search
3. Bayesian Optimisation

There are numerous different hyperparameters available for XGBoost. A complete listing of them can be found here: https://xgboost.readthedocs.io/en/stable/parameter.html. For the purpose of our work here, I'll only consider tuning the following:

* n_estimators
* eta / learning_rate
* max_depth
* reg_lambda / L2 regularization
* reg_alpha / L1 regularization

For this demonstration, we will create a toy dataset using scikit-learn's *make_classification*. As such, I'll be optimizing the XGBClassifier algorithm.

We can start by importing the packages necessary here:

In [1]:
# imports
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import uniform, poisson

Now let's create our dataset, and do a train-test split:

In [2]:
# load in and prepare data
X, y = make_classification(n_samples=1000, 
                           n_features=100, 
                           n_informative=60,
                           n_classes=2, 
                           weights=[0.4,0.6],
                           random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Baseline

To be able to measure the effects of our tuning, let's first measure how well XGBoost does on the test set with all default values:

In [3]:
# fit a model with default parameters
clf = XGBClassifier(random_state=42)
clf.fit(X_train, y_train)

# compute performance on test set
y_pred = clf.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test, y_pred):.2f}')
print(f"precision score: {precision_score(y_test, y_pred):.2f}")
print(f"recall score: {recall_score(y_test, y_pred):.2f}")
print(f"f1 score: {f1_score(y_test, y_pred):.2f}")

accuracy score: 0.83
precision score: 0.81
recall score: 0.91
f1 score: 0.86


## Grid Search

Brute force approach to hyperparameter tuning. Each parameter configuration will be validated using 5-fold Cross-Validation. Afterwards, the best model will be selected, and tested against our held-out test set.

In [4]:
# setup parameter space
parameters = {
    'n_estimators':[50, 100, 200, 500, 1000],
    'learning_rate': [0.1, 0.3, 0.6, 0.8, 1.0],
    'max_depth':[1, 3, 6, 10],
    'reg_alpha':[0.0, 0.1, 0.5, 1.0],
    'reg_lambda':[0.1, 0.5, 1.0, 1.5]
}

# create an instance of the grid search object
g = GridSearchCV(XGBClassifier(random_state=42), parameters, cv=5, n_jobs=-1)

# conduct grid search over the parameter space
start_time = time.time()
g.fit(X_train, y_train)
duration = time.time() - start_time

# show best parameter configuration found for classifier
cls_params = g.best_params_
cls_params

/opt/homebrew/Caskroom/miniforge/base/envs/blog/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 1000,
 'reg_alpha': 0.0,
 'reg_lambda': 1.0}

In [5]:
# compute performance on test set
model = g.best_estimator_
y_pred = model.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test,y_pred):.2f}')
print(f'precision score: {precision_score(y_test,y_pred):.2f}')
print(f'recall score: {recall_score(y_test,y_pred):.2f}')
print(f'f1 score: {f1_score(y_test,y_pred):.2f}')
print(f'computation time: {duration:.2f}')

accuracy score: 0.81
precision score: 0.78
recall score: 0.92
f1 score: 0.85
computation time: 306.76


## Randomized Search

We can do hyperparameter tuning through random sampling from a probability distribution. Each parameter configuration will be validated using 5-fold Cross-Validation. Afterwards, the best model will be selected, and tested against our held-out test set.

In [6]:
# setup parameter space
parameters = {
    'n_estimators':poisson(mu=500),
    'learning_rate': uniform(),
    'max_depth':poisson(mu=6),
    'reg_alpha':uniform(loc=0, scale=2),
    'reg_lambda':uniform(loc=0, scale=2)
}

# create an instance of the randomized search object
r = RandomizedSearchCV(XGBClassifier(random_state=42), parameters, cv=5, n_iter=100, random_state=42, n_jobs=-1)

# conduct grid search over the parameter space
start_time = time.time()
r.fit(X_train,y_train)
duration = time.time() - start_time

# show best parameter configuration found for classifier
cls_params2 = r.best_params_
cls_params2

/opt/homebrew/Caskroom/miniforge/base/envs/blog/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'learning_rate': 0.12063587110060081,
 'max_depth': 5,
 'n_estimators': 521,
 'reg_alpha': 0.16351806389774382,
 'reg_lambda': 1.7471572482135544}

In [7]:
# compute performance on test set
model = r.best_estimator_
y_pred = model.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test,y_pred):.2f}')
print(f'precision score: {precision_score(y_test,y_pred):.2f}')
print(f'recall score: {recall_score(y_test,y_pred):.2f}')
print(f'f1 score: {f1_score(y_test,y_pred):.2f}')
print(f'computation time: {duration:.2f}')

accuracy score: 0.83
precision score: 0.80
recall score: 0.94
f1 score: 0.86
computation time: 31.84


## Bayesian Optimization

The final method we'll try takes advantage of Bayes theorem for hyperparameter tuning. Like before, the parameter space is defined by a set of probability distributions, in the form of priors. Care will be needed when selecting these prior distributions. Each parameter configuration will be validated using 5-fold Cross-Validation. Afterwards, the best model will be selected, and tested against our held-out test set.

In [8]:
# setup parameter space
parameters = {
    'n_estimators':Integer(50,1000,prior='uniform'),
    'learning_rate': Real(0.0001,1,prior='log-uniform'),
    'max_depth':Integer(1,10,prior='uniform'),
    'reg_alpha':Real(0.0001,2,prior='log-uniform'),
    'reg_lambda':Real(0.0001,2,prior='log-uniform')
}

# create an instance of the bayesian search object
b = BayesSearchCV(XGBClassifier(random_state=42), parameters, cv=5, n_iter=5, random_state=42, n_jobs=-1)

# conduct randomized search over the parameter space
start_time = time.time()
b.fit(X_train,y_train)
duration = time.time() - start_time

# show best parameter configuration found for classifier
cls_params3 = b.best_params_
cls_params3

OrderedDict([('learning_rate', 0.15783879853890564),
             ('max_depth', 5),
             ('n_estimators', 550),
             ('reg_alpha', 0.11975791012918251),
             ('reg_lambda', 0.7717824087995475)])

In [9]:
# compute performance on test set
model = b.best_estimator_
y_pred = model.predict(X_test)
print(f'accuracy score: {accuracy_score(y_test,y_pred):.2f}')
print(f'precision score: {precision_score(y_test,y_pred):.2f}')
print(f'recall score: {recall_score(y_test,y_pred):.2f}')
print(f'f1 score: {f1_score(y_test,y_pred):.2f}')
print(f'computation time: {duration:.2f}')

accuracy score: 0.83
precision score: 0.81
recall score: 0.93
f1 score: 0.87
computation time: 11.31
